In [ ]:
%pip install lark
%pip install networkx
from lark import Lark

## LHS parser ##

parsing of the pattern sent as lhs, into a networkX graph representing the template to search.

The module converts the declerative constraints regarding the properties of the nodes and edges in the LHS, to imperative functions that are checked together with the 'condition' parameter

## grammar

In [ ]:
"blabla"

In [ ]:

lhs_parser = Lark(r"""
    patterns: pattern (";" pattern)*
            
    pattern: [vertex (connection vertex)*]

    vertex: named_vertex attributes
          | index_vertex attributes
          | ANONYMUS attributes

    named_vertex: [a-zA-Z0-9]* #any string with numbers and letters only
    index_vertex: named_vertex "<" NATURAL_NUMBER ("," NATURAL_NUMBER)* ">"
    ANONYMUS: "_"

    connection: "-" [attributes "-"] ">"
              | multi_connection

    attributes: "\[" attribute (, #allow optional \n here, for imperative syntax)
                              attribute)* "\]"

    multi_connection: "-" NATURAL_NUMBER "+" [attributes "-"] "->" # explicit minimal constraint
                    | "-" NATURAL_NUMBER [attributes "-"] "->" # deterministic number of connections

    attribute: attr_name [":" type] ["=" value] 

    attr_name: #lark-imported
    type: #lark-imported #escaped string or word
    value: #imported?

    NATURAL_NUMBER: #imported?

    %import common.ESCAPED_STRING
    %import common.WS #CHANGE to allow \n in the imperative option.
    %ignore WS

    """, start='patterns')

## Transformer
The transformer is designed to return the networkX graph representing the patterns.

For each branch, the appropriate method will be called with the children of the branch as its argument, and its return value will replace the branch in the tree.

In [ ]:
from lark import Transformer
class lhsTransformer(Transformer):
    def attribute(self, attr_name, type, value):
        # return attr_name, constraints are handled in other transformer.
        pass

    def multi_connection(self, number, attributes): # +
        # return the list of attributes(strings), number of duplications,
        #   and FALSE (indicating that the connection is not deterministic)

        # renewed: return the list of attributes(strings), add a special attribute to denote number of duplications,
        #   and FALSE (indicating that the connection is not deterministic)
        pass
    def multi_connection(self, number, attributes ): # no +
        # return the list of attributes(strings), number of duplications,
        #  and TRUE (indicating that the connection is deterministic)

        # renewed: return the list of attributes(strings), add a special attribute to denote number of duplications,
        #   and TRUE (indicating that the connection is not deterministic)
        pass

    def attributes(self, *attributes):
        # return a packed list of the attribute names.
        pass

    def connection(self, attributes_list, num_duplications, is_deterministic): #multiconnection
        # return the packed list of attributes received, num_duplications, is_deterministic
        pass

    def connection(self, attributes): #
        # return the packed list of attributes received, num_duplications = 1, is_deterministic = True
        pass

    def ANONYMUS(self): #
        # return a dedicated name for anonymus (string), and an empty list.
        pass

    def sub_vertex(self, main_name, *numbers):
        # return the main name of the vertex, and a list of the indices specified.
        pass
    
    def named_vertex(self, __):
        # return the main name of the vertex, and an empty list.
        pass

    def vertex(self, name, indices_list, attributes_list):
        # return arguments
        pass

    def pattern(self, vertex, *connections_to_vertex):
        # 1) unpack lists of vertices and connections.
        # 2) create a networkX graph:
            # if there is a special attribute with TRUE, dumplicate the connection __number__ times.

        pass

    def patterns(self,):

        pass

## Type and constant value checking
The transformer is designed to collect the node type and constant node value constraints, such that they are added to the 'condition' parameter to be checked later.

This transformer works on a copy of the tree to keep it intact.

In [ ]:
class collectTypeConstraints(Transformer):
    def attribute(self, attr_name, type, value):
        # return a mapping from attr_name - > required type and value
        pass

    def attributes(self, *attributes):
        # return a packed list of the attribute mappings.
        pass

    def vertex(self, name, indices_list, attributes_list):
        # same as lhsTransformer
        pass

    def pattern(self, vertex, *connections_to_vertex):
        # return arguments
        pass

    def patterns(self, *patterns):
        # unpack lists of vertices and connections.
        def typeCondition(Match):
            # for every vertex in vertex list:
                # create full_vertex_name by the attached indices list
                # for every attr, type, name required for the vertex:
                    # constructor = getName(type) - get the constructor for the type
                    # 1) check that the required type and value match together.
                    # try:
                    #     instance = constructor(value)
                    # Except:
                        # flag = False: value does not match the type.

                    # 2) check that the value constraint holds
                    # if getattr(instance, __eq__) == None:
                        # flag = False. the type must implement __eq__
                    # if not (instance == match[full_vertex_name][attr])

                    # no need to check the type constraint(?), if the value fits. (python)

            # TODO: perform the same iterations in the connections list.

            #return flag and condition(Match)
            pass

        return typeCondition #sent as a module output and replaces condition.
        pass

Apply the Transformers

In [ ]:
required_syntax =  """
a -> b

a -[x:int = ...]-> b

a -> b[x:int = ...]

a -> b -6+[weight:int]-> c -> d[value:int]
d<0> -> e
d<5> -> e

b -+-> d[value:int]
d<0> -7-> e
e<0,5> -> _

b[ \
value: str = \"hello\", \
id: int \
]

b -[
...
]-> c 

"""